In [1]:
import warnings
warnings.filterwarnings("ignore")

from experiment import Exp

import torch
import yaml
import os

In [2]:
### Parameter for training

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   

args.to_save_path     = "../Run_logs"           
args.window_save_path = "../Sliding_window"
args.root_path        = "../datasets"

args.drop_transition  = False
# Nomalization of the data, depends on dataset!
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"

args.batch_size       = 32 # Reduce if not enough graphic memory
args.shuffle          = True # Give NN more variety to learn
args.drop_last        = True # Drop last batch if smaller than batch size
args.train_vali_quote = 0.90 # 90% training, 10% validation                     

# Training setting 
args.train_epochs            = 60

# According to ShaSpec paper
args.learning_rate           = 0.001 # 10^(-3)
args.learning_rate_patience  = 20 # Reduce learning rate every 20 epochs
args.learning_rate_factor    = 0.1 # Reduce learning rate by 10%

args.weight_decay            = 0.01 # 10^(-2)

args.early_stop_patience     = 15 # Stop after 15 epochs without improvement

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0 # Use first GPU
args.use_multi_gpu           = False # Use multiple GPUs

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

In [3]:
# Get content of folder args.window_save_path
files = os.listdir(args.window_save_path)

In [4]:
### Parameter for data

args.seed                             = 999 # Seed for reproducibility

args.data_name                        = "dsads"

# Wavelet calculates frequence parameters from time series data, don't need that
args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 6

# Augmentation difference
args.difference          =  False

# Augmentation filtering
args.filtering           =  False

# Augmentation  magnitude
args.magnitude           =  False
args.weighted_sampler    = False

# MIXUP
args.mixup_probability  = 1
args.mixup_alpha  = 0.5

# Random Augmentation
args.random_augmentation_prob = 1
args.random_augmentation_config = {"jitter": True, 
                                   "moving_average": True,
                                   "magnitude_scaling": True,
                                   "magnitude_warp": True,
                                   "magnitude_shift": True,
                                   "time_warp": True,
                                   "window_warp": True,
                                   "window_slice": True,
                                   "random_sampling": True,
                                   "slope_adding": True
                                   }
random_augmentation_nr = 0
for key in args.random_augmentation_config.keys():
    if args.random_augmentation_config[key]:
        random_augmentation_nr = random_augmentation_nr+1
args.random_augmentation_nr = random_augmentation_nr
args.max_aug = 3

args.pos_select       = None
args.sensor_select    = None

args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
    
config_file = open('configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
args.num_modalities  =  config["num_modalities"]
args.miss_rates       =  config["miss_rates"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# Input information
args.c_in            =  config["num_channels"]
# For ShaSpec we want the number of channels per modality
args.c_in_per_mod    =  config["num_channels"] // args.num_modalities

args.f_in            =  1
            

In [5]:
### Parameter for the model

args.model_type = "shaspec"

args.miss_rate = args.miss_rates[3]

args.activation = "ReLU"
args.shared_encoder_type = "concatenated"

# Ablation studies
args.ablate_shared_encoder = True
args.ablate_missing_modality_features = False

exp = Exp(args)

Use GPU: cuda:0
================  ShaSpec Model Configuration  ================
Number of total modalities:  5
Selected miss rate:  0.6
Number of available modalities:  2
Ablate shared encoder:  True
Ablate missing modality features:  False
Build the ShaSpec model!
Done!
Parameter: 325909
Set the seed as:  999


In [6]:
exp.model

model_builder(
  (model): ShaSpec(
    (specific_encoders): ModuleList(
      (0-1): 2 x SpecificEncoder(
        (be): BaseEncoder(
          (conv_layers): Sequential(
            (0): Conv2d(1, 64, kernel_size=(5, 1), stride=(2, 1))
            (1): ReLU()
            (2): Conv2d(64, 64, kernel_size=(5, 1), stride=(2, 1))
            (3): ReLU()
            (4): Conv2d(64, 64, kernel_size=(5, 1), stride=(2, 1))
            (5): ReLU()
            (6): Conv2d(64, 64, kernel_size=(5, 1), stride=(2, 1))
            (7): ReLU()
          )
          (sa): SelfAttention(
            (query): Conv1d(64, 8, kernel_size=(1,), stride=(1,), bias=False)
            (key): Conv1d(64, 8, kernel_size=(1,), stride=(1,), bias=False)
            (value): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
          )
          (fc_fusion): Linear(in_features=320, out_features=128, bias=True)
        )
      )
    )
    (decoder): Decoder(
      (flatten): Flatten(start_dim=1, end_dim=-1)
     

In [7]:
exp.train()

================  model_shaspec_data_dsads_seed_999_miss_rate_0.6_ablate_shared_encoder_True_ablate_missing_modality_features_False  ================
================  Load all the data  ================
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================  LOCV Mode  ================
================  4 CV folds in total  ================
================  CV 0  ================
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.00303951 0.00308642 0.0030303  0.003125   0.0030581  0.00307692
 0.00308642 0.00304878 0.00307692 0.00316456 0.00315457 0.00303951
 0.0030303  0.00310559 0.00307692 0.00307692 0.00311526 0.0031746
 0.00308642]
Train data number:  6156
Validation data number:  684
Test data number:  2280
================  Build the shaspec model  ================
================  ShaSpec Model Configuration  ================
Number of 

Training Process [epochs]:   0%|          | 0/60 [00:00<?, ?it/s]

Epoch: 1 cost time: 290.2840099334717


Training Process [epochs]:   2%|▏         | 1/60 [05:22<5:16:47, 322.16s/it]

VALI: Epoch: 1, Steps: 193 | Train Loss: nan  Vali Loss: nan Vali Accuracy: 0.0453216  Vali weighted F1: 0.0039300  Vali macro F1 0.0045639  Vali micro F1 0.0453216
Validation loss decreased (inf --> nan).  Saving model ...
Epoch: 2 cost time: 289.0458514690399


...

VALI: Epoch: 12, Steps: 193 | Train Loss: nan  Vali Loss: nan Vali Accuracy: 0.0453216  Vali weighted F1: 0.0039300  Vali macro F1 0.0045639  Vali micro F1 0.0453216
new best score!!!!
Validation loss decreased (nan --> nan).  Saving model ...
new best score!!!!
Epoch: 13 cost time: 290.1310076713562


Training Process [epochs]:  22%|██▏       | 13/60 [1:09:29<4:11:16, 320.78s/it]

VALI: Epoch: 13, Steps: 193 | Train Loss: nan  Vali Loss: nan Vali Accuracy: 0.0453216  Vali weighted F1: 0.0039300  Vali macro F1 0.0045639  Vali micro F1 0.0453216
new best score!!!!
Validation loss decreased (nan --> nan).  Saving model ...
new best score!!!!
Epoch: 14 cost time: 288.09931206703186


Training Process [epochs]:  23%|██▎       | 14/60 [1:14:49<4:05:44, 320.52s/it]

VALI: Epoch: 14, Steps: 193 | Train Loss: nan  Vali Loss: nan Vali Accuracy: 0.0453216  Vali weighted F1: 0.0039300  Vali macro F1 0.0045639  Vali micro F1 0.0453216
new best score!!!!
Validation loss decreased (nan --> nan).  Saving model ...
new best score!!!!
